# regression ultimate code

In [1]:
import numpy as np
import argparse

import os
import math
from math import sin

import json

import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

from xgboost import XGBRegressor

import seaborn as sns

import matplotlib
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
matplotlib.rc( 'savefig', facecolor = 'white' )
from matplotlib import pyplot
%matplotlib inline

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms, datasets, models
from torchsummary import summary
torch.manual_seed(0)

i_seed = 0

import sys
sys.path.append('../') # add this line so Data and data are visible in this file
sys.path.append('../../') # add this line so Data and data are visible in this file
sys.path.append('../PatchWand/') # add this line so Data and data are visible in this file

# from PatchWand import *
from plotting_tools import *
from setting import *
from models import *
from models_CNN import *
from evaluate import *

from stage3_preprocess import *
from training_util import *
from dataset_util import *
from dataIO import *
from stage4_regression import *

from importlib import reload
%load_ext autoreload
%autoreload 2

In [2]:
print(torch.version.cuda)

11.3


In [3]:
parser = argparse.ArgumentParser(description='SpO2_estimate')
parser.add_argument('--input_folder', metavar='input_folder', help='input_folder',
                    default='../')
parser.add_argument('--output_folder', metavar='output_folder', help='output_folder',
                    default='../')
parser.add_argument('--training_params_file', metavar='training_params_file', help='training_params_file',
                    default='training_params_list.json')


# checklist 3: comment first line, uncomment second line
args = parser.parse_args(['--input_folder', '../../data/stage3/', 
#                           '--output_folder', '../../data/stage4/ML_regression/',
                          '--output_folder', '../../data/stage4/VE_regression/',
                          '--training_params_file', 'training_params_ML.json',
#                           '--training_params_file', 'training_params_baseline.json',
                         ])
# args = parser.parse_args()
print(args)

Namespace(input_folder='../../data/stage3/', output_folder='../../data/stage4/VE_regression/', training_params_file='training_params_ML.json')


In [4]:
inputdir = args.input_folder
outputdir = args.output_folder
training_params_file = args.training_params_file

if not os.path.exists(outputdir):
    os.makedirs(outputdir)

    

# get training params and dataloaders

In [5]:
# training_params['list_feature']

In [6]:
# stage3_dict['list_feature']

In [7]:
with open(training_params_file) as json_file:
    training_params_list = json.load(json_file)

for training_params in training_params_list:
    # include device in training_params
#     device = torch.device('cuda:{}'.format(int(training_params['cuda_i'])) if torch.cuda.is_available() else 'cpu')
#     training_params['device'] = device


    training_params['CV_config'] = {
        'subject_id': 101,
        'task_id': 1,
    }
    stage3_dict = data_loader('stage3_dict', inputdir).item()
    training_params['list_signal'] = stage3_dict['list_signal']
    training_params['list_feature'] = stage3_dict['list_feature']
    print( training_params['list_feature'] )
    training_params['list_output'] = stage3_dict['list_output']
    training_params['list_meta'] = stage3_dict['list_meta']
    training_params['FS_RESAMPLE_DL'] = stage3_dict['FS_RESAMPLE_DL']
    training_params['subject_ids'] = stage3_dict['subject_ids']
    training_params['task_ids'] = stage3_dict['task_ids']
    
#     input_CV = '../../data/stage3/113/CV2/'
#     dataloaders, dataset_sizes = get_loaders(input_CV, training_params)
    dataloaders, dataset_sizes = get_loaders(inputdir, training_params)
    print('data dimensions are:', dataloaders['train'].dataset.data.shape)

    data_dimensions = dataloaders['train'].dataset.__getitem__(0)[0].size()
    training_params['data_dimensions'] = list(data_dimensions)
    
    sweep_name = training_params['sweep_name'] 
    

training_params = training_params_list[0]

['VE_cosmed', 'HR_patch', 'HR_patchnormed', '0.00Hz', '3.91Hz', '7.81Hz', '11.72Hz', '15.62Hz', '19.53Hz', '23.44Hz', 'weight', 'height', 'gender', 'age', 'BMR', 'BMI']
data dimensions are: (559, 3, 6000)


In [8]:
training_params['feature_names'] = ['VE_cosmed']

In [9]:
# training_params['feature_names']  = ['HR_patch']

In [10]:
# feature_train.shape

## HP from Mobashir's paper
- learning rate = 0.05, 
- max_depth=10, 
- subsample=0.6, 
- colsample_bytree = 0.7, 
- n_estimators = 100, 
- min_child_weight = 2, 
- gamma = 0.3.

In [11]:



# feature_train.shape
# sig = feature_train[0,0,:]
# t_sig = np.arange(sig.shape[0])/training_params['FS_RESAMPLE_DL']

# # arange(sig.shape[0])

# plt.plot(t_sig, sig)
# sig_smoothed = get_smooth(sig, N=int(training_params['FS_RESAMPLE_DL']*20))
# plt.plot(t_sig, sig_smoothed)


# downsample_factor = training_params['FS_RESAMPLE_DL']*10 # take a sample every 10 sec (6 datapoints)
# plt.scatter(t_sig[downsample_factor//2:][::downsample_factor], sig_smoothed[downsample_factor//2:][::downsample_factor])


In [17]:
outputdir_numeric = outputdir + 'numeric_results/'
if outputdir_numeric is not None:
    if not os.path.exists(outputdir_numeric):
        os.makedirs(outputdir_numeric)
    
outputdir_modelout = outputdir + 'model_output/'
if outputdir_modelout is not None:
    if not os.path.exists(outputdir_modelout):
        os.makedirs(outputdir_modelout)
        
        
        

In [16]:
df_performance_train = pd.DataFrame()
df_performance_val = pd.DataFrame()

df_outputlabel_train = pd.DataFrame()
df_outputlabel_val = pd.DataFrame()



# training_params['feature_names'] = ['HR_patch']
# training_params['feature_names'] = ['HR_patch', 'weight', 'height', 'gender']
# training_params['feature_names'] = ['VE_cosmed', 'weight']
# training_params['feature_names'] = ['VE_cosmed']
training_params['output_names'] = ['EE_cosmed']

for subject_id in training_params['subject_ids']:

    training_params['CV_config']['subject_id'] = subject_id

    dataloaders, dataset_sizes = get_loaders(inputdir, training_params)
    
#     i_feature = training_params['feature_names'].index('HR_patch')
#     i_label = training_params['output_names'].index('EE_cosmed')

    
    if training_params['model_name']=='LinearRegression':
        model = LinearRegression()
    elif training_params['model_name']=='XGBRegressor':
        model = XGBRegressor(learning_rate=0.05, max_depth=10, subsample=0.6, colsample_bytree=0.7, n_estimators=100, min_child_weight=2, gamma=0.3)
    
    feature_train = dataloaders['train'].dataset.feature[:,:,0]
    feature_val = dataloaders['val'].dataset.feature[:,:,0]
    
    # zero mean unit variance
    feature_mean = np.mean(feature_train, axis=0)
    feature_std = np.std(feature_train, axis=0)

    feature_train = (feature_train-feature_mean)/feature_std
    feature_val = (feature_val-feature_mean)/feature_std

    
    label_train = dataloaders['train'].dataset.label.squeeze()
    label_val = dataloaders['val'].dataset.label.squeeze()
    
    model = model.fit(feature_train, label_train)
    label_est_train = model.predict(feature_train)
    label_est_val = model.predict(feature_val)
    
    label_train = label_train.squeeze()
    label_val = label_val.squeeze()
    label_est_train = label_est_train.squeeze()
    label_est_val = label_est_val.squeeze()
    
    
    # get performance df for training and testing dataset
    df_performance_train = df_performance_train.append( get_df_performance(label_train, label_est_train, subject_id, training_params['output_names'][0]), ignore_index=True )
    df_performance_train.to_csv(outputdir_numeric+'df_performance_train.csv', index=False)

    df_outputlabel_train = df_outputlabel_train.append(
        pd.DataFrame( {
        'label_est': label_est_train,
        'label': label_train,
        'CV': [subject_id]*label_train.shape[0],
        'task': [training_params['output_names'][0]]*label_train.shape[0]
        }), ignore_index=True )

    df_outputlabel_train.to_csv(outputdir_numeric+'df_outputlabel_train.csv', index=False)

    df_performance_val = df_performance_val.append( get_df_performance(label_val, label_est_val, subject_id, training_params['output_names'][0]), ignore_index=True )
    df_performance_val.to_csv(outputdir_numeric+'df_performance_val.csv', index=False)

    df_outputlabel_val = df_outputlabel_val.append(
        pd.DataFrame( {
        'label_est': label_est_val,
        'label': label_val,
        'CV': [subject_id]*label_val.shape[0],
        'task': [training_params['output_names'][0]]*label_val.shape[0]
        }), ignore_index=True )

    df_outputlabel_val.to_csv(outputdir_numeric+'df_outputlabel_val.csv', index=False)
    
    


In [ ]:
# # plot performance training and testing dataset
# plot_regression(df_outputlabel_train[task], df_performance_train[task], task, fig_name='regression_train_{}'.format(task), show_plot=False, outputdir=outputdir+'model_output/')
# plot_BA(df_outputlabel_train[task], task, fig_name='BA_train_{}'.format(task), show_plot=False, outputdir=outputdir+'model_output/')

# plot_regression(df_outputlabel_val[task], df_performance_val[task], task, fig_name='regression_val_{}'.format(task), show_plot=False, outputdir=outputdir+'model_output/')
# plot_BA(df_outputlabel_val[task], task, fig_name='BA_val_{}'.format(task), show_plot=False, outputdir=outputdir+'model_output/')

# plot_output(df_outputlabel_train[task], fig_name = 'outputINtime_train_', show_plot=True, outputdir=outputdir+'model_output/')
# plot_output(df_outputlabel_val[task], fig_name = 'outputINtime_val_',  show_plot=True, outputdir=outputdir+'model_output/')


In [18]:
# plot performance training and testing dataset
task = training_params['output_names'][0]
plot_regression(df_outputlabel_train, df_performance_train, task, fig_name='regression_train', show_plot=False, outputdir=outputdir_modelout)
plot_BA(df_outputlabel_train, task, fig_name='BA_train', show_plot=False, outputdir=outputdir_modelout)
plot_output(df_outputlabel_train, task, fig_name = 'outputINtime_train_', show_plot=False, outputdir=outputdir_modelout)

plot_regression(df_outputlabel_val, df_performance_val, task, fig_name='regression_val', show_plot=False, outputdir=outputdir_modelout)
plot_BA(df_outputlabel_val, task, fig_name='BA_val', show_plot=False, outputdir=outputdir_modelout)
plot_output(df_outputlabel_val, task, fig_name = 'outputINtime_val_',  show_plot=False, outputdir=outputdir_modelout)


In [19]:
# 

In [20]:
task_name = task.split('_')[0]

label_range = [my_floor(df_outputlabel_val['label'].values.min()), my_ceil(df_outputlabel_val['label'].values.max())]

N_sub = len(df_outputlabel_val['CV'].unique())
N_samples = df_outputlabel_val.shape[0]
t_dur = N_samples*3/60


PCC = get_PCC(df_outputlabel_val['label'].values, df_outputlabel_val['label_est'].values)
Rsquared = get_CoeffDeterm(df_outputlabel_val['label'].values, df_outputlabel_val['label_est'].values)
MAE, MAE_std = get_MAE(df_outputlabel_val['label'].values, df_outputlabel_val['label_est'].values)
RMSE = get_RMSE(df_outputlabel_val['label'].values, df_outputlabel_val['label_est'].values)
MAPE, MAPE_std = get_MAPE(df_outputlabel_val['label'].values, df_outputlabel_val['label_est'].values)

title_str = '{} range: {:.1f}-{:.1f} {}'.format(task.split('_')[0], label_range[0], label_range[1], unit_dict[task_name])
textstr = 'RMSE={:.2f} {}\nMAE={:.2f} {}\nMAPE={:.2f} {}\nPCC={:.2f}\nR2={:.2f}\nN_sub={}\nN_samples={}\nduration={:.2f} min'.format(
    RMSE, unit_dict[task_name], MAE, unit_dict[task_name],MAPE*100, '%',
    PCC, Rsquared,
    N_sub, N_samples, t_dur)

In [21]:
print(textstr)

RMSE=0.49 kcals/min
MAE=0.38 kcals/min
MAPE=20.00 %
PCC=0.91
R2=0.82
N_sub=15
N_samples=599
duration=29.95 min


In [ ]:
data_saver(training_params, 'training_params', outputdir)

In [ ]:
sys.exit()

In [ ]:
#     for task in training_params['tasks']:
#         label_est_val = CV_dict['performance_dict_val']['out_dict'][task]
#         label_val = CV_dict['performance_dict_val']['label_dict'][task]

#         label_est_train = CV_dict['performance_dict_train']['out_dict'][task]
#         label_train = CV_dict['performance_dict_train']['label_dict'][task]
        
#         # get performance df for training and testing dataset
#         df_performance_train = df_performance_train.append( get_df_performance(label_train, label_est_train, subject_id, task), ignore_index=True )
#         df_performance_train.to_csv(outputdir+'df_performance_train.csv', index=False)

#         df_outputlabel_train = df_outputlabel_train.append(
#             pd.DataFrame( {
#             'label_est': label_est_train,
#             'label': label_train,
#             'CV': [subject_id]*label_train.shape[0],
#             'task': [task]*label_train.shape[0]
#             }), ignore_index=True )
        
#         df_outputlabel_train.to_csv(outputdir+'df_outputlabel_train.csv', index=False)

#         df_performance_val = df_performance_val.append( get_df_performance(label_val, label_est_val, subject_id, task), ignore_index=True )
#         df_performance_val.to_csv(outputdir+'df_performance_val.csv', index=False)
        
#         df_outputlabel_val = df_outputlabel_val.append(
#             pd.DataFrame( {
#             'label_est': label_est_val,
#             'label': label_val,
#             'CV': [subject_id]*label_val.shape[0],
#             'task': [task]*label_val.shape[0]
#             }), ignore_index=True )
        
#         df_outputlabel_val.to_csv(outputdir+'df_outputlabel_val.csv', index=False)

#         # plot performance training and testing dataset
#         plot_regression(df_outputlabel_train, df_performance_train, task, fig_name='regression_train', show_plot=False, outputdir=outputdir)
#         plot_BA(df_outputlabel_train, task, fig_name='BA_train', show_plot=False, outputdir=outputdir)
        
#         plot_regression(df_outputlabel_val, df_performance_val, task, fig_name='regression_val', show_plot=False, outputdir=outputdir)
#         plot_BA(df_outputlabel_val, task, fig_name='BA_val', show_plot=False, outputdir=outputdir)

# #     sys.exit()

In [ ]:
# plot_regression(df_outputlabel_val, df_performance_val, task, show_plot=True, outputdir=outputdir)


In [ ]:
# plot_BA(df_outputlabel_val, task, show_plot=True, outputdir=outputdir)
